In [3]:
#Initial imports for testing
import pandas as pd
from flask import jsonify
import numpy as np
import requests
import os
import psycopg2
from dotenv import load_dotenv
from datetime import datetime

In [4]:
#Load environment variables
load_dotenv(override=True)
host = os.getenv('RDS_HOSTNAME')
port = os.getenv('RDS_PORT')
db = os.getenv('RDS_DB_NAME')
usern = os.getenv('RDS_USERNAME')
passw = os.getenv('RDS_PASSWORD')

In [5]:
def pull_repo(owner, name):
    variables = {'owner': owner, 'name': name}
    response = run_query(repo_query, variables)
    data = response.json()['data']['repository']

    data['repoName'] = data['name']
    data['stars'] = data['stars']['totalCount']
    data['owner'] = data['owner']['login']
    data['primaryLanguage'] = data['primaryLanguage']['name']
    data['totalIssues'] = data['totalIssues']['totalCount']
    data['openIssues'] = data['openIssues']['totalCount']
    data['closedIssues'] = data['closedIssues']['totalCount']
    data['totalPRs'] = data['totalPRs']['totalCount']
    data['openPRs'] = data['openPRs']['totalCount']
    data['mergedPRs'] = data['mergedPRs']['totalCount']
    data['closedPRs'] = data['closedPRs']['totalCount']
    data['vulnerabilityAlerts'] = data['vulnerabilityAlerts']['totalCount']

    if (data['mergedPRs'] + data['closedPRs'] != 0):
        data['PRacceptanceRate'] = data['mergedPRs'] / (data['mergedPRs'] +
                                                        data['closedPRs'])
    else:
        data['PRacceptanceRate'] = None
    data['createdAt'] = datetime.strptime(data['createdAt'],
                                          DATE_FORMAT)
    data['updatedAt'] = datetime.strptime(data['updatedAt'],
                                          DATE_FORMAT)
    data['ageInDays'] = (datetime.now().date() -
                         data['createdAt'].date()).days
    data['starsPerDay'] = data['stars'] / data['ageInDays']
    data['forksPerDay'] = data['forks'] / data['ageInDays']
    data['PRsPerDay'] = data['totalPRs'] / data['ageInDays']
    data['issuesPerDay'] = data['totalIssues'] / data['ageInDays']

    return data

def run_query(query, variables):
    r = requests.post(URL,
                      headers={'Authorization': 'token ' + SECRET, },
                      json={'query': query,
                            'variables': variables
                            })
    return r

def summarize_PRs(pr_df):
    data = {}
    pr_df = pd.DataFrame(pr_df, index=[0])
    if pr_df.empty:
        data['uniquePRauthors'] = 0
        data['medianOpenPRhrsAge'] = None
        data['medianPRhrsToClose'] = None
        data['medianPRhrsToMerge'] = None
    else:
        pr_df['author'] = [author.get('login') if author is not None else ''
                           for author in pr_df['author']]
        pr_df['createdAt'] = pd.to_datetime(pr_df['createdAt'],
                                            format=DATE_FORMAT)
        pr_df['closedAt'] = pd.to_datetime(pr_df['closedAt'],
                                           format=DATE_FORMAT)

        data['uniquePRauthors'] = pr_df['author'].nunique()

        openPRs = pr_df['state'] == 'OPEN'
        if openPRs.empty:
            data['medianOpenPRhrsAge'] = None
        else:
            openPRsecsAge = (datetime.now() -
                             pr_df['createdAt']).dt.total_seconds()[openPRs]
            data['medianOpenPRhrsAge'] = openPRsecsAge.median()/SECS_PER_HOUR

        closedPRs = pr_df['state'] == 'CLOSED'
        if closedPRs.empty:
            data['medianPRhrsToClose'] = None
        else:
            PRsecsToClose = (pr_df['closedAt'] -
                             pr_df['createdAt']).dt.total_seconds()[closedPRs]
            data['medianPRhrsToClose'] = PRsecsToClose.median()/SECS_PER_HOUR

        mergedPRs = pr_df['state'] == 'MERGED'
        if mergedPRs.empty:
            data['medianPRhrsToMerge'] = None
        else:
            PRsecsToMerge = (pr_df['closedAt'] -
                             pr_df['createdAt']).dt.total_seconds()[mergedPRs]
            data['medianPRhrsToMerge'] = PRsecsToMerge.median()/SECS_PER_HOUR

    return data

In [6]:
URL = 'https://api.github.com/graphql'
SECRET = os.getenv('SECRET')
DATE_FORMAT = '%Y-%m-%dT%H:%M:%SZ'
SECS_PER_HOUR = 3600

In [7]:
variables = {'owner': 'scikit-learn', 'name': 'scikit-learn'}
response = run_query(initial_PR_query, variables)
data = response.json()

In [31]:
conn = psycopg2.connect(database=db, user=usern,
                        password=passw, host=host,
                        port=port)

In [32]:
create_table = '''
CREATE Table PullRequests(RepoName VARCHAR,
                          OwnerName VARCHAR,
                          ID VARCHAR,
                          State VARCHAR,
                          CreatedAt VARCHAR,
                          ClosedAt VARCHAR,
                          Title VARCHAR,
                          BodyText VARCHAR,
                          Author VARCHAR,
                          Participants INT,
                          Comments INT,
                          Reactions INT,
                          Commits INT,
                          ChangedFiles INT,
                          Additions INT,
                          Deletions INT)
'''

In [33]:
curs = conn.cursor()
curs.execute(create_table)

In [ ]:
for i in range(len(data2)):
    test_insert = ("INSERT INTO PullRequests VALUES (" +
               "'" + str(variables['name']) +"'" +  ", " +
               "'" + str(variables['owner']) +"'" +  ", " + 
               "'" + str(data2[i]['id']) +"'" +  ", " +
               "'" + str(data2[i]['state']) +"'" +  ", " + 
               "'" + str(data2[i]['createdAt']) + "'" + ", " +
               "'" + str(data2[i]['closedAt']) + "'" +  ", " +
               "'" + str(data2[i]['title'].replace("'","")) +"'" +  ", " + 
               "'" + str(data2[i]['bodyText'].replace("'","")) +"'" +  ", " +
               "'" + str(data2[i]['author']['login']) +"'" +  ", " + 
               "'" + str(data2[i]['participants']['totalCount']) +"'" +  ", " + 
               "'" + str(data2[i]['comments']['totalCount']) +"'" +  ", " + 
               "'" + str(data2[i]['reactions']['totalCount'])+"'" + ", " +
               "'" + str(data2[i]['commits']['totalCount']) +"'" +  ", " + 
               "'" + str(data2[i]['changedFiles']) +"'" +  ", " +
               "'" + str(data2[i]['additions']) +"'" +  ", " + 
               "'" + str(data2[i]['deletions']) + "'" +  ")")
    curs.execute(test_insert)
    print(curs.fetchall())

In [ ]:
print(curs.fetchall())

In [12]:
data2 = data['data']['repository']['pullRequests']['nodes']


In [93]:
data2

{'id': 'MDExOlB1bGxSZXF1ZXN0NjU0',
 'state': 'MERGED',
 'createdAt': '2010-09-01T13:06:00Z',
 'closedAt': '2010-09-02T14:10:16Z',
 'title': "more on LARS (we're getting there)",
 'bodyText': '',
 'author': {'login': 'agramfort'},
 'participants': {'totalCount': 3},
 'comments': {'totalCount': 2},
 'reactions': {'totalCount': 0},
 'commits': {'totalCount': 9},
 'changedFiles': 7,
 'additions': 153,
 'deletions': 77}

In [82]:
str(data2['id'])

'MDExOlB1bGxSZXF1ZXN0NjU0'

In [ ]:
test_insert = ("INSERT INTO PullRequests VALUES (" +
               "'" + str(variables['name']) +"'" +  ", " +
               "'" + str(variables['owner']) +"'" +  ", " + 
               "'" + str(data2['id']) +"'" +  ", " +
               "'" + str(data2['state']) +"'" +  ", " + 
               "'" + str(data2['createdAt'] + "'" + ", " +
               "'" + str(data2['closedAt'] + "'" +  ", " +
               "'" + str(data2['title'].replace("'","")) +"'" +  ", " + 
               "'" + str(data2['bodyText']) +"'" +  ", " +
               "'" + str(data2['author']['login']) +"'" +  ", " + 
               "'" + str(data2['participants']['totalCount']) +"'" +  ", " + 
               "'" + str(data2['comments']['totalCount']) +"'" +  ", " + 
               "'" + str(data2['reactions']['totalCount'])+"'" + ", " +
               "'" + str(data2['commits']['totalCount']) +"'" +  ", " + 
               "'" + str(data2['changedFiles']) +"'" +  ", " +
               "'" + str(data2['additions']) +"'" +  ", " + 
               "'" + str(data2['deletions']) + "'" +  ")")

In [82]:
test_insert

"INSERT INTO PullRequests (RepoName, OwnerName, Id, State, CreatedAt, ClosedAt, Title, BodyText, Author, Participants, Comments, Commits, ChangedFiles, Additions, Deletions)  VALUES ('scikit-learn', 'scikit-learn', 'MDExOlB1bGxSZXF1ZXN0NjU0', 'MERGED', '2010-09-01 13:06:00', '2010-09-02 14:10:16', 'more on LARS (were getting there)', '', 'agramfort', '3', '2', '9', '7', '153', '77')"

In [83]:
curs.execute(test_insert)

In [84]:
curs3 = conn.cursor()
curs3.execute('SELECT * FROM PullRequests')

In [85]:
print(curs3.fetchall())

[('scikit-learn', 'scikit-learn', 'MDExOlB1bGxSZXF1ZXN0NjU0', 'MERGED', '2010-09-01 13:06:00', '2010-09-02 14:10:16', 'more on LARS (were getting there)', '', 'agramfort', 3, 2, None, 9, 7, 153, 77)]


In [38]:
data['name'] = variables['name']
data['owner'] = variables['owner']

In [19]:
data_df = pd.DataFrame(data, index=[0])

In [1]:
repo_query = '''
    query ($owner: String!, $name: String!){
        repository(owner: $owner, name: $name) {
            name
            owner {
                login
                }
            description
            primaryLanguage {
                name
                }
            stars: stargazers {
                totalCount
                }
            forks: forkCount
            totalIssues: issues {
            totalCount
                }
            openIssues: issues (states: [OPEN]) {
                totalCount
                }
            closedIssues: issues (states: [CLOSED]) {
                totalCount
                }
            vulnerabilityAlerts {
                totalCount
                }
            totalPRs: pullRequests {
                totalCount
                }
            openPRs: pullRequests (states: [OPEN]) {
                totalCount
                }
            mergedPRs: pullRequests (states: [MERGED]) {
                totalCount
                }
            closedPRs: pullRequests (states: [CLOSED]) {
                totalCount
                }
            createdAt
            updatedAt
            diskUsage
        }
    }
    '''

initial_PR_query = '''
    query ($owner: String!, $name: String!){
        repository(owner: $owner, name: $name) {
            pullRequests(first: 10) {
                pageInfo {
                    endCursor
                    hasNextPage
                }
                nodes {
                    id
                    state
                    createdAt
                    closedAt
                    title
                    bodyText
                    author {
                        login
                    }
                    participants {
                        totalCount
                    }
                    comments {
                        totalCount
                    }
                    reactions {
                        totalCount
                    }
                    commits {
                        totalCount
                    }
                    changedFiles
                    additions
                    deletions
                }
            }
        }
    }
    '''

cont_PR_query = '''
    query ($owner: String!, $name: String!, $cursor: String!) {
        repository(owner: $owner, name: $name) {
            pullRequests(first: 50 after: $cursor) {
                pageInfo {
                    endCursor
                    hasNextPage
                }
                nodes {
                    id
                    state
                    createdAt
                    closedAt
                    title
                    bodyText
                    author {
                        login
                    }
                    participants {
                        totalCount
                    }
                    comments {
                        totalCount
                    }
                    reactions {
                        totalCount
                    }
                    commits {
                        totalCount
                    }
                    changedFiles
                    additions
                    deletions
                }
            }
        }
    }
    '''